In [430]:
#Make a dataframe with all options prices based on black scholes
#Data pulled from the fed, etherscan and t3 ethvol index
#Can add things such as better strikes, vol smile, etc

In [431]:
import numpy as np
import pandas as pd
import scipy
from scipy.stats import norm
import datetime

In [432]:
eth = "/Users/nick/Downloads/ethvolatility.csv"
ethvix = pd.read_csv(eth)
tre = "/Users/nick/Downloads/DGS102.csv"
treas = pd.read_csv(tre)
ethp = "/Users/nick/Downloads/EtPrice.csv"
ethprice = pd.read_csv(ethp)


In [433]:
#cutting both frames to the first day which ETH-VIX data is available
treas['DATE'] = pd.to_datetime(treas['DATE'])
treas.set_index('DATE', drop=True, inplace=True)
treas = treas.loc['2020-4-15':]

ethprice.drop(columns = ['UnixTimeStamp'], axis=1,inplace=True)
ethprice.rename(columns={'Date(UTC)':'DATE'},inplace =True)
ethprice['DATE'] = pd.to_datetime(ethprice['DATE'])
ethprice.set_index('DATE', drop=True, inplace=True)
ethprice = ethprice.loc['2020-4-15':]


In [434]:
treas['DGS10'] = treas['DGS10'].replace('^.$',np.nan,regex=True) #replace a problematic row
treas['DGS10'] = pd.to_numeric(treas['DGS10']) #converting to float

In [435]:
ethvix

,"Date;""Ethereum Volatility Index"""
0,2020-04-15 00:00:00;105.48
1,2020-04-16 00:00:00;103.27
2,2020-04-17 00:00:00;102.55
3,2020-04-18 00:00:00;99.87
4,2020-04-19 00:00:00;101.69
...,...
808,2022-07-13 00:00:00;100.52
809,2022-07-14 00:00:00;98.92
810,2022-07-15 00:00:00;96.56
811,2022-07-16 00:00:00;102.58


In [436]:
#yuck!

In [437]:
ethvix[['DATE','VIX']] = ethvix[ethvix.columns[0]].str.split(pat=';',expand=True)
ethvix.drop(ethvix.columns[0],axis=1, inplace=True)
ethvix['DATE'] = pd.to_datetime(ethvix['DATE'])
ethvix.set_index('DATE', drop=True, inplace=True)
ethvix['VIX'] = pd.to_numeric(ethvix['VIX']) #converting to float
ethvix['VIX'] = ethvix['VIX']/100 #divide by 100 so we can put it into black-scholes


In [438]:
#almost done cleaning, need to fill weekends for treasuries and other missing values

idx = pd.date_range('04-15-2020', '07-16-2022')
treas = treas.reindex(idx, fill_value=np.nan)
ethvix = ethvix.reindex(idx, fill_value=np.nan)
treas['DGS10'].ffill(inplace=True)
ethvix['VIX'].ffill(inplace=True)

In [439]:
chaindata = treas.join(ethvix)
chaindata = chaindata.join(ethprice)
chaindata

,DGS10,VIX,Value
2020-04-15,0.63,105.48,152.85
2020-04-16,0.61,103.27,172.58
2020-04-17,0.65,102.55,170.98
2020-04-18,0.65,99.87,187.81
2020-04-19,0.65,101.69,180.34
...,...,...,...
2022-07-12,2.96,103.14,1037.37
2022-07-13,2.91,100.52,1114.57
2022-07-14,2.96,98.92,1192.56
2022-07-15,2.96,96.56,1231.23
